### 파이프라인(Retriever-Reader) 평가

1. pipepath경로에 run.py의 최하단 부분에서 저장했던 파이프라인 경로를 적어줍니다.
2. readerpath경로에 reader모델(또는 run.py에서 훈련시켰던)을 저장한 경로를 적어줍니다.
3. 무지성 실행 고고

In [1]:
from haystack.schema import EvaluationResult

pipepath = '/opt/ml/input/level2_nlp_mrc-nlp-01/output/2023-06-13-16:51/pipeline_result'
readerpath = '/opt/ml/input/code/models'

saved_eval_result = EvaluationResult.load(pipepath)

In [2]:
metrics = saved_eval_result.calculate_metrics()
print(f'Retriever - Recall (single relevant document): {metrics["Retriever"]["recall_single_hit"]}')
print(f'Retriever - Recall (multiple relevant documents): {metrics["Retriever"]["recall_multi_hit"]}')
print(f'Retriever - Mean Reciprocal Rank: {metrics["Retriever"]["mrr"]}')
print(f'Retriever - Precision: {metrics["Retriever"]["precision"]}')
print(f'Retriever - Mean Average Precision: {metrics["Retriever"]["map"]}')

print(f'Reader - F1-Score: {metrics["Reader"]["f1"]}')
print(f'Reader - Exact Match: {metrics["Reader"]["exact_match"]}')

Retriever - Recall (single relevant document): 0.6416666666666667
Retriever - Recall (multiple relevant documents): 0.6416666666666667
Retriever - Mean Reciprocal Rank: 0.5078439153439154
Retriever - Precision: 0.10541666666666669
Retriever - Mean Average Precision: 0.48237764550264545
Reader - F1-Score: 0.3989636752136752
Reader - Exact Match: 0.3416666666666667


In [99]:
from haystack.nodes import FARMReader
from haystack.nodes import BM25Retriever
from haystack.pipelines import ExtractiveQAPipeline
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore()
retriever = BM25Retriever(document_store=document_store)

In [5]:

reader = FARMReader(model_name_or_path=readerpath)

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever) 

Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}


In [6]:
pipeline.print_eval_report(saved_eval_result)

================== Evaluation Report ==================
                   Pipeline Overview
                      Query
                        |
                        |
                      Retriever
                        |
                        | recall_single_hit: 0.762
                        |
                      Reader
                        |
                        | exact_match:   0.4
                        | exact_match_top_1:   0.3
                        | f1:   0.5
                        | f1_top_1: 0.343
                        | num_examples_for_eval: 2.4e+02
                        | num_examples_for_eval_top_1: 2.4e+02
                        |
                      Output

                Wrong Retriever Examples
Query: 
 	일본의 대학 입시는 며칠간 진행되는가?
Gold Document Ids: 
 	mrc-0-001073
Metrics: 
 	recall_multi_hit: 0.0
 	recall_single_hit: 0.0
 	precision: 0.0
 	map: 0.0
 	mrr: 0.0
 	ndcg: 0.0
Documents: 
 	context: 바이쿨라 전투 이후 하스드루발 바르카는 이탈리아로 떠났고 히스파니아의 카르타고군은 

In [ ]:
#> SAS 지표

eval_labels = document_store.get_all_labels_aggregated(drop_negative_labels=True, drop_no_answers=True)


advanced_eval_result = pipeline.eval(
    labels=eval_labels, 
    params={"Retriever": {"top_k": 10}},
    sas_model_name_or_path="cross-encoder/stsb-roberta-large"
)

metrics = advanced_eval_result.calculate_metrics()
print(metrics["Reader"]["sas"])

In [27]:
import json
import pandas as pd

with open('/opt/ml/input/level2_nlp_mrc-nlp-01/output/2023-06-13-16:51/outputs.json', 'r') as f:
    data = json.load(f)

df_read = pd.DataFrame(data, index=[0])
df_read_sorted = df_read.sort_index(axis=1)
df_read_sorted

,mrc-0-000001,mrc-0-000014,mrc-0-000016,mrc-0-000021,mrc-0-000028,mrc-0-000030,mrc-0-000032,mrc-0-000033,mrc-0-000037,mrc-0-000049,...,mrc-1-001721,mrc-1-001724,mrc-1-001727,mrc-1-001729,mrc-1-001735,mrc-1-001769,mrc-1-001775,mrc-1-001797,mrc-1-001802,mrc-1-001829
0,'강연옥',유대인들의 집을 만들어줄 것을 요구하였다. 1936년 필 위원회는 팔레스타인 지구를...,로맨틱 코미디,1862년,김 대령,사세충렬문,국방부 보고서,하난야,미켈란젤로,전국통일,...,김수환,노스햄프턴 교회,수영공원,수리물리학,마리아 팔코네티,잉글랜드 왕국,이마가와야카타(今川館),강화부,호메이니,화흡


In [28]:
with open('/opt/ml/input/level2_nlp_mrc-nlp-01/data/output(base).json', 'r') as f:
    data = json.load(f)

df_base = pd.DataFrame(data, index=[0])
df_base_sorted = df_base.sort_index(axis=1)
df_base_sorted

,mrc-0-000001,mrc-0-000014,mrc-0-000016,mrc-0-000021,mrc-0-000028,mrc-0-000030,mrc-0-000032,mrc-0-000033,mrc-0-000037,mrc-0-000049,...,mrc-1-001721,mrc-1-001724,mrc-1-001727,mrc-1-001729,mrc-1-001735,mrc-1-001769,mrc-1-001775,mrc-1-001797,mrc-1-001802,mrc-1-001829
0,'강연옥',와라이 족,경음악이 보급되기 시작했다. 1930년대에 이르러 라틴 음악인,1862년,김 대령,응접실,루이스 곳샬크,하난야,쿠로다 칸이치,게임플레이,...,이경희,노스햄프턴 교회,수영공원,수리물리학,마리아 팔코네티,발해,포룸 로마눔,폐암,호메이니,화흡


In [75]:
df_join = pd.concat([df_read_sorted, df_base_sorted], axis=0)

In [76]:
df = df_join.T
df = df.reset_index()
df.columns = ['id', 'read', 'base']

In [77]:
df_test = pd.read_csv('/opt/ml/input/data/test.csv')
df_test = df_test.set_index('id')

,id,read,base
0,mrc-0-000001,'강연옥','강연옥'
1,mrc-0-000014,유대인들의 집을 만들어줄 것을 요구하였다. 1936년 필 위원회는 팔레스타인 지구를...,와라이 족
2,mrc-0-000016,로맨틱 코미디,경음악이 보급되기 시작했다. 1930년대에 이르러 라틴 음악인
3,mrc-0-000021,1862년,1862년
4,mrc-0-000028,김 대령,김 대령
...,...,...,...
595,mrc-1-001769,잉글랜드 왕국,발해
596,mrc-1-001775,이마가와야카타(今川館),포룸 로마눔
597,mrc-1-001797,강화부,폐암
598,mrc-1-001802,호메이니,호메이니


In [91]:
df_join = df.join(df_test, on='id')

In [92]:
df_join = df_join[['question', 'base', 'read']]

In [93]:
df_join

,question,base,read
0,안영애를 본보기로 만든 <당의 참된 딸>이라는 가극에서 극 중 여자 주연의 이름은?,'강연옥','강연옥'
1,섬 정부가 영국 정부가 지시내리기전에 섬에서 모두 쫓아낸 민족은?,와라이 족,유대인들의 집을 만들어줄 것을 요구하였다. 1936년 필 위원회는 팔레스타인 지구를...
2,1930년대 흑인음악으로 백인들의 열렬한 사랑을 받았던 재즈의 한 분야는?,경음악이 보급되기 시작했다. 1930년대에 이르러 라틴 음악인,로맨틱 코미디
3,브레킨리지가 루이지애나주 배턴루지를 얻는 데 실패한 해는?,1862년,1862년
4,요원들의 마지막 임무 대상은 누구인가?,김 대령,김 대령
...,...,...,...
595,마가렛의 모친은 어떤 나라의 왕족이었나?,발해,잉글랜드 왕국
596,장면이 가족과 떨어져 거처를 옮긴 곳은?,포룸 로마눔,이마가와야카타(今川館)
597,이경언이 사망한 장소는?,폐암,강화부
598,레바논 전투에서 헤즈볼라는 누구에 대한 지지를 표명했나요?,호메이니,호메이니


In [96]:
df_n = df_join[df_join['base'] != df_join['read']]

In [114]:
df_n

,question,base,read
1,섬 정부가 영국 정부가 지시내리기전에 섬에서 모두 쫓아낸 민족은?,와라이 족,유대인들의 집을 만들어줄 것을 요구하였다. 1936년 필 위원회는 팔레스타인 지구를...
2,1930년대 흑인음악으로 백인들의 열렬한 사랑을 받았던 재즈의 한 분야는?,경음악이 보급되기 시작했다. 1930년대에 이르러 라틴 음악인,로맨틱 코미디
5,수사들이 전쟁의 대가로 지은 건물은?,응접실,사세충렬문
6,미군이 충원되지 않으면 서울이 전쟁 시작 후 2주만에 함락될 것이라 밝힌 문서는?,루이스 곳샬크,국방부 보고서
8,피렌체파의 대미를 수놓은 사람은?,쿠로다 칸이치,미켈란젤로
...,...,...,...
588,유럽에서 두 번째로 큰 국가에는 몇 개의 국립공원이 존재하는가?,독일,중국
590,선생님에게 창씨개명을 하지 않겠다고 한 사람은?,이경희,김수환
595,마가렛의 모친은 어떤 나라의 왕족이었나?,발해,잉글랜드 왕국
596,장면이 가족과 떨어져 거처를 옮긴 곳은?,포룸 로마눔,이마가와야카타(今川館)


In [115]:
retriever.retrieve(query="이경언이 사망한 장소는?	", top_k=20)

[<Document: {'content': '장례와 묘소의 위치는 미상이고, 시호와 작호도 추서되지 못하였다. 또한 사후 양자나 봉사손도 지명되지 않았다. 1821년(순조 21)에 사망한 그의 생모 역시 의금부를 대신해서 강화부에서 검험한 뒤, 비밀리에 시신을 처리하였으나 역시 장소는 미상이다.\n\n1849년 철종 즉위 후 은언군 가족 및 관련자들, 상계군 내외 등이 복작되거나 복권되고, 전산군부인 이씨, 이원경, 이욱 등 작위 미수여자에게 작위가 추서되었지만 그는 증직과 증작을 받지 못하고 잊혀졌다. 철종 즉위 직후, 그리고 철종때와 고종 때 은언군 이인 관련 자료들, 상계군 이담이 홍국영에 의해 정조의 양자로 추대되었다가 역적으로 몰린 것에 대한 관련 자료 등을 모두 삭제하고, 천주교에 대한 상산군부인 송마리아와 평산군부인 신마리아 등에 대한 조선왕조실록, 승정원일기, 일성록 기사 및 관련 기록, 체포, 국문 등에 대한 기록들을 대량으로 세초, 먹줄, 파기할 때 그에 관련된 기사들도 대부분 인멸, 세초되었다.\n\n그의 생모와 형제 철득 등은 선원보에서 누락되었고, 천첩인지 양첩인지 여부를 알 수 없다.  이들 은언\n군의 일부 서출들은 1970년대 조선왕조실록과 각종 조선시대 문헌을 한글로 번역하는 과정에서 그 존재가 알려지게 되었다.', 'content_type': 'text', 'score': 0.805738119079524, 'meta': {'_split_id': 0, 'author': None, 'domain': None, 'html': None, 'corpus_source': '위키피디아', 'title': '이성득 (조선 왕족)', 'document_id': 44441, 'url': None}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'd8256c4849b96fa09dd172fef9760f24'}>,
 <Document: {'content': '천수가 남만즈쿠리  으로 복원되었다. 이는